Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [95]:
from typing import List, Tuple
from collections import namedtuple
import random
from tqdm import tqdm
import heapq



In [96]:
action = namedtuple('Action', ['pos1', 'pos2'])

Helper Functions

In [98]:
# Find the position of the zero (empty tile)
def find_empty_pos (state, DIM):
  found = False
  empty_pos = (0,0)
  for x in range(DIM):
    for y in range(DIM):
      if state[x][y] == 0:
        empty_pos = (x, y)
        found = True
        break
    if found == True:
      break
  return empty_pos

def available_actions(state, DIM):
    empty_pos = find_empty_pos(state,DIM)
    x, y = empty_pos
    actions = []

    if x > 0:
        actions.append(action((x, y), (x - 1, y))) #up
    if x < DIM - 1:
        actions.append(action((x, y), (x + 1, y))) #down
    if y > 0:
        actions.append(action((x, y), (x, y - 1))) #left
    if y < DIM - 1:
        actions.append(action((x, y), (x, y + 1))) #right
    return actions


def do_action(state, action):
    state_list = [list(row) for row in state] #change to lists because tuple is immutable
    x1, y1 = action.pos1
    x2, y2 = action.pos2
    # Swap the tiles in the lists
    state_list[x1][y1], state_list[x2][y2] = state_list[x2][y2], state_list[x1][y1]
    new_state = tuple(tuple(row) for row in state_list) #create the new tuple
    return new_state

#how far a state from the solution state
def manhattan_distance(state, goal_state,DIM):
    distance = 0
    for x in range(DIM):
        for y in range(DIM):
            value = state[x][y]
            if value != 0:
                goal_x, goal_y = divmod(value - 1, DIM) # calculate the goal position of x,y
                distance += abs(x - goal_x) + abs(y - goal_y) #calculate the distance between the corrent pos to the goal pos
    return distance


def is_visited(myset, state):
  return state in myset

def initialize_state(state,RANDOMIZE_STEPS,dim):
  for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    actions = available_actions(state,dim)
    state = do_action(state, random.choice(actions))
  return state

def create_goal_state(dim):
    numbers = [i for i in range(1, dim * dim)] + [0]
    goal_state = tuple([tuple(numbers[i * dim:(i + 1) * dim]) for i in range(dim)])
    return goal_state

def print_state(state):
    for row in state:
        print(' '.join(f"{num:2}" for num in row))
    print()




A-STAR algorithm

In [99]:
def A_star(first_state, goal_state, puzzle_dim):
    priority_heap = []
    visited_states = set()
    dict_heap = dict()
    g_first_cost = 0
    f_first_cost = g_first_cost + manhattan_distance(first_state, goal_state, puzzle_dim) #g+h
    g_h_costs = {first_state: (g_first_cost ,f_first_cost)}
    first_node = [f_first_cost, first_state]
    heapq.heappush(priority_heap, first_node) #add to the heap the first state
    dict_heap[first_state] = first_node
    index = 1


    while len(priority_heap) != 0:
      current = heapq.heappop(priority_heap)
      current_f = current[0]
      current_state = current[1]

      if current_state == goal_state: #end game
            print(f'The number of actions is: {index}')
            print(f'The g_cost is: {g_h_costs[current_state][0]}')
            print_state(current_state)
            return

      if index % 1000 == 0:
        print(f'actions number: {index}')
        print_state(current_state)

      visited_states.add(current_state) #add to the already visited set

      for action in available_actions(current_state, puzzle_dim):
        new_state = do_action(current_state, action)

        if is_visited(visited_states,new_state): #if we already visit this state-ignore
          continue

        index = index+1

        h_new_state = 0
        if new_state not in priority_heap:
          h_new_state = manhattan_distance(new_state, goal_state, puzzle_dim)
          g_new_state = g_h_costs[current_state][0] + 1
          f_new_state = g_new_state + h_new_state
          g_h_costs[new_state] = (g_new_state,h_new_state)

          new_node = [f_new_state, new_state]
          heapq.heappush(priority_heap, new_node) #add the new state to the heap
          dict_heap[new_state] = new_node

        #the state is already in the heap
        else:
          h_new_state = g_h_costs[new_state][1]
          g_new_state = g_h_costs[current_state][0] + 1
          f_new_state = g_new_state + h_new_state

          if g_new_state < g_h_costs[new_state][0]: #update the shortest way
            g_h_costs[new_state] = (g_new_state,h_new_state)
            dict_heap[new_state][0] = f_new_state



In [101]:
PUZZLE_DIM = 3
RANDOMIZE_STEPS = 1000


# create and print the goal state
goal_state = create_goal_state(PUZZLE_DIM)
print('The goal state is:')
print_state(goal_state)

# crate and print the randomized state
first_state = initialize_state(goal_state,RANDOMIZE_STEPS,PUZZLE_DIM)
print('\nThe initial state is:')
print_state(first_state)


solution_path = A_star(first_state, goal_state, PUZZLE_DIM)


The goal state is:
 1  2  3
 4  5  6
 7  8  0



Randomizing: 100%|██████████| 1000/1000 [00:00<00:00, 153496.94it/s]


The initial state is:
 7  2  0
 8  6  3
 4  1  5

The number of actions is: 895
The g_cost is: 22
 1  2  3
 4  5  6
 7  8  0

